![Illustration of silhouetted heads](mentalhealth.jpg)

Does going to university in a different country affect your mental health? A Japanese international university surveyed its students in 2018 and published a study the following year that was approved by several ethical and regulatory boards.

The study found that **international students have a higher risk of mental health difficulties than the general population**. Explore the `students` data using PostgreSQL to find out if this is true and see if the length of stay is a contributing factor.

Here is a data description of the fields you may find helpful. The full dataset is in one table with 50 fields and, according to the survey, 268 records. Each row is a student.

| Field Name    | Description                                      | 
| ------------- | ------------------------------------------------ |
| inter_dom     | Types of students                                |
| japanese_cate | Japanese language proficiency                    | 
| english_cate  | English language proficiency                     |
| academic      | Current academic level                           | 
| age           | Current age of student                           |
| stay          | Current length of stay in years                  |
| todep         | Total score of depression (PHQ-9 test)           |
| tosc          | Total score of social connectedness (SCS test)   |
| toas          | Total score of Acculturative Stress (ASISS test) |

Your task will be to do the following exploratory analysis:

- Count the number of all records, and all records per student type
- Filter the data to see how it differs between the student types
- Find the summary statistics of the diagnostic tests for all students
- Summarize the data for international students
- See if length of stay impacts the test scores

In [58]:
-- View the full dataset
SELECT *
FROM students
LIMIT 5;

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,SEA,Male,Grad,24,4,5,Long,3,Average,5,High,,Yes,No,No,No,0,Min,34,23,9,11,8,11,2,27,91,5,5,6,3,2,1,4,1,3,4,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
1,Inter,SEA,Male,Grad,28,5,1,Short,4,High,4,High,,No,No,No,No,2,Min,48,8,7,5,4,3,2,10,39,7,7,7,4,4,4,4,1,1,1,NaN,Yes,Yes,Yes,No,No,No,No,No,No,No,No
2,Inter,SEA,Male,Grad,25,4,6,Long,4,High,4,High,Yes,Yes,No,No,No,2,Min,41,13,4,7,6,4,3,14,51,3,3,3,1,1,2,1,1,1,1,NaN,No,No,No,No,No,No,No,No,No,No,No
3,Inter,EA,Female,Grad,29,5,1,Short,2,Low,3,Average,No,No,No,No,No,3,Min,37,16,10,10,8,6,4,21,75,5,5,5,5,5,2,2,2,4,4,NaN,Yes,Yes,Yes,Yes,Yes,No,No,No,No,No,No
4,Inter,EA,Female,Grad,28,5,1,Short,1,Low,3,Average,Yes,No,No,No,No,3,Min,37,15,12,5,8,7,4,31,82,5,5,5,2,5,2,5,5,4,4,NaN,Yes,Yes,Yes,No,Yes,No,Yes,Yes,No,No,No


## 1. Exploring the Dataset:
#### Start by exploring the dataset and understanding the data we are working with: 
#### First, counting the number of records in the dataset:

In [59]:
SELECT COUNT(*) as total_records
FROM students;

,total_records
0,286


#### There are, in fact 286 records as specified.
#### Next, counting the number of records for each student type:

In [60]:
SELECT 
	inter_dom,
	COUNT(*) AS count_inter_dom
FROM students
GROUP BY inter_dom
ORDER BY inter_dom ASC;

,inter_dom,count_inter_dom
0,,18
1,Dom,67
2,Inter,201


#### There are 67 domestic students, 201 international students, and 18 students unspecified.

## 2. Create a filter to explore how the data is different in each student type

#### Exploring the 18 unassigned type_of_students to see what that's about:

In [61]:
SELECT inter_dom, COUNT(*) AS number_of_records
FROM students
WHERE inter_dom NOT LIKE 'D%' AND inter_dom NOT LIKE 'I%'
GROUP BY inter_dom

,inter_dom,number_of_records
0,NaN,18


#### The remaining rows just have the inter_dom unspecified, which don't help our determination of whether Japanese students have lower mental health than the general population. Thus, this will need to be removed from the dataset when analyzing data.

#### Next, analyze the type of students under the 'Inter' category. See where the students are from:

In [62]:
SELECT region, COUNT(*) AS region_count
FROM students
WHERE inter_dom = 'Inter'
GROUP BY region
ORDER BY region_count DESC;

,region,region_count
0,SEA,122
1,EA,48
2,SA,18
3,Others,11
4,JAP,2


#### The majority of the international students are from SEA, or Southeast Asia. It's interesting that two students are in the international category but are from JAP. Let us take a quick glance at those two:

In [63]:
SELECT *
FROM students 
WHERE inter_dom = 'Inter'
	AND region = 'JAP';

,inter_dom,region,gender,academic,age,age_cate,stay,stay_cate,japanese,japanese_cate,english,english_cate,intimate,religion,suicide,dep,deptype,todep,depsev,tosc,apd,ahome,aph,afear,acs,aguilt,amiscell,toas,partner,friends,parents,relative,profess,phone,doctor,reli,alone,others,internet,partner_bi,friends_bi,parents_bi,relative_bi,professional_bi,phone_bi,doctor_bi,religion_bi,alone_bi,others_bi,internet_bi
0,Inter,JAP,Female,Under,21,3,10,Long,5,High,3,Average,No,No,Yes,Yes,Other,13,Mod,32,9,4,10,8,4,2,13,50,1,1,3,1,1,1,1,1,1,1,1,No,No,No,No,No,No,No,No,No,No,No
1,Inter,JAP,Male,Under,19,2,1,Short,4,High,4,High,No,No,No,No,No,2,Min,40,24,12,15,12,6,6,30,105,3,3,3,3,3,3,3,3,3,3,3,No,No,No,No,No,No,No,No,No,No,No


## 3. Find out the summary statistics of the diagnostic tests for all students
#### Compute the aggregrate functions minimum, maximum, and average, and round the averages to two decimal places and use aliases to keep the output clean. The diagnostic test fields are: 
* PHQ-9 for scoring depression (todep): higher is worse
* SCS test for scoring social connectedness (tosc): to assess the extent to which persons feel connected to others in their surrounding social area, higher is worse
* ASISS test for scoring acculturative stress (toas): the stress that emerges from conflicts when individuals must adjust to a new culture of the host society, higher is worse
#### (aliases should be min_phq, max_phq, and avg_phq & repeat this structure for the other two tests)

In [64]:
SELECT
	MIN(todep) AS min_phq,
	MAX(todep) AS max_phq,
	ROUND(AVG(todep),2) AS avg_phq,
	MIN(tosc) AS min_scs,
	MAX(tosc) AS max_scs,
	ROUND(AVG(toas),2) AS avg_scs,
	MIN(toas) AS min_asiss,
	MAX(toas) AS max_asiss,
	ROUND(AVG(toas),2) AS avg_asiss
FROM students;

,min_phq,max_phq,avg_phq,min_scs,max_scs,avg_scs,min_asiss,max_asiss,avg_asiss
0,0,25,8.19,8,48,72.38,36,145,72.38


## 4. Summarize the data for international students only and local students only and compare

In [65]:
SELECT
	inter_dom,
	MIN(todep) AS min_phq,
	MAX(todep) AS max_phq,
	ROUND(AVG(todep),2) AS avg_phq,
	MIN(tosc) AS min_scs,
	MAX(tosc) AS max_scs,
	ROUND(AVG(toas),2) AS avg_scs,
	MIN(toas) AS min_asiss,
	MAX(toas) AS max_asiss,
	ROUND(AVG(toas),2) AS avg_asiss
FROM students
WHERE inter_dom LIKE 'D%' OR inter_dom LIKE 'I%' -- to remove the null values
GROUP BY inter_dom;

,inter_dom,min_phq,max_phq,avg_phq,min_scs,max_scs,avg_scs,min_asiss,max_asiss,avg_asiss
0,Inter,0,25,8.04,11,48,75.56,36,145,75.56
1,Dom,0,23,8.61,8,48,62.84,36,112,62.84


#### Observations:
* Average PHQ-9 score for depression (avg_phq): roughly the same, slightly higher in domestic students
* Average SCS score for social connectedness (avg_scs): greater in international students  
* Average ASISS score for acculturative stress (avg_assiss): greater in international students

Slightly higher score indicating higher depression in domestic students without any other factors, but slightly higher SCS and ASISS scores indicating higher stress from social connectedness and acculturative stress.

## 5. See the impact of the length of stay
#### See how the length of stay of an international student impacts the average diagnostic scores. Order the results by descending order of the length of stay. The order of your fields should be average_phq, average_scs, and average_as

In [66]:
SELECT stay, 
	ROUND(AVG(todep),2) AS average_phq,
	ROUND(AVG(toas),2) AS average_scs,
	ROUND(AVG(toas),2) AS average_as
FROM students
WHERE inter_dom = 'Inter'
GROUP BY stay
ORDER BY stay DESC;

,stay,average_phq,average_scs,average_as
0,10,13.00,50.00,50.00
1,8,10.00,65.00,65.00
2,7,4.00,45.00,45.00
3,6,6.00,58.67,58.67
4,5,0.00,91.00,91.00
5,4,8.57,87.71,87.71
6,3,9.09,78.00,78.00
7,2,8.28,77.67,77.67
8,1,7.48,72.80,72.80


#### Interestingly, the longer the international students stayed, the higher their PHQ score would be. On the other hand, the students staying longest had the lowest core in SCS and ASISS, suggesting better social connectedness and less acculturative stress over time.